<a href="https://colab.research.google.com/github/Snowdenstyll/Lottery/blob/main/Keno_Lottery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [30]:
df = pd.read_csv("2024.csv")

In [31]:
df.head()
df.tail()
df.info()
df.describe()
df.drop(['PlayDate', 'AP'], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324 entries, 0 to 323
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   PlayDate  324 non-null    object
 1   AP        324 non-null    object
 2   N01       324 non-null    int64 
 3   N02       324 non-null    int64 
 4   N03       324 non-null    int64 
 5   N04       324 non-null    int64 
 6   N05       324 non-null    int64 
 7   N06       324 non-null    int64 
 8   N07       324 non-null    int64 
 9   N08       324 non-null    int64 
 10  N09       324 non-null    int64 
 11  N10       324 non-null    int64 
 12  N11       324 non-null    int64 
 13  N12       324 non-null    int64 
 14  N13       324 non-null    int64 
 15  N14       324 non-null    int64 
 16  N15       324 non-null    int64 
 17  N16       324 non-null    int64 
 18  N17       324 non-null    int64 
 19  N18       324 non-null    int64 
 20  N19       324 non-null    int64 
 21  N20       324 no

In [32]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [33]:
# All our games
number_of_rows = df.values.shape[0]
# Amount of games we need to take into consideration for prediction
window_length = 5
number_of_features = df.values.shape[1]

X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)


In [34]:
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [35]:
# Recurrent Neural Netowrk (RNN) with Long Short Term Memory (LSTM)
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
#batch_size = 40
batch_size = 32
ltsm_units = 256
dropout = 0.2

# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(ltsm_units,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(dropout))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(ltsm_units,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(dropout))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(ltsm_units,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(ltsm_units,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a fourth Dropout layer
model.add(Dropout(dropout))
# Adding the first output layer
model.add(Dense(70))
# Adding the last output layer
model.add(Dense(number_of_features))

In [36]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

model.compile(optimizer=Adam(learning_rate=0.001), loss ='mse', metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.89, random_state=89)

# Train the model
#history = model.fit(X_train, y_train, batch_size=batch_size, epochs=200, verbose=2)
history = model.fit(X_train, y_train, batch_size=30, epochs=600, verbose=2)

Epoch 1/600
2/2 - 15s - loss: 0.8654 - accuracy: 0.0000e+00 - 15s/epoch - 7s/step
Epoch 2/600
2/2 - 0s - loss: 0.8120 - accuracy: 0.0857 - 30ms/epoch - 15ms/step
Epoch 3/600
2/2 - 0s - loss: 0.7549 - accuracy: 0.0571 - 32ms/epoch - 16ms/step
Epoch 4/600
2/2 - 0s - loss: 0.7028 - accuracy: 0.0571 - 31ms/epoch - 15ms/step
Epoch 5/600
2/2 - 0s - loss: 0.6569 - accuracy: 0.0571 - 31ms/epoch - 15ms/step
Epoch 6/600
2/2 - 0s - loss: 0.5766 - accuracy: 0.0286 - 42ms/epoch - 21ms/step
Epoch 7/600
2/2 - 0s - loss: 0.5559 - accuracy: 0.0571 - 31ms/epoch - 15ms/step
Epoch 8/600
2/2 - 0s - loss: 0.5585 - accuracy: 0.1714 - 30ms/epoch - 15ms/step
Epoch 9/600
2/2 - 0s - loss: 0.5735 - accuracy: 0.2571 - 32ms/epoch - 16ms/step
Epoch 10/600
2/2 - 0s - loss: 0.5288 - accuracy: 0.1714 - 30ms/epoch - 15ms/step
Epoch 11/600
2/2 - 0s - loss: 0.4800 - accuracy: 0.2000 - 28ms/epoch - 14ms/step
Epoch 12/600
2/2 - 0s - loss: 0.4609 - accuracy: 0.1143 - 33ms/epoch - 17ms/step
Epoch 13/600
2/2 - 0s - loss: 0.459

In [37]:
loss, accuracy = model.evaluate(X, y, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

10/10 [==============================] - 2s 8ms/step - loss: 1.3323 - accuracy: 0.1630
Test Loss: 1.3323029279708862, Test Accuracy: 0.1630094051361084
9/9 [==============================] - 0s 7ms/step - loss: 1.4954 - accuracy: 0.0669
Test Loss: 1.495447039604187, Test Accuracy: 0.06690140813589096


In [38]:
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)

In [39]:
import datetime
import pytz

# Get the current time in a specific timezone
tz = pytz.timezone('America/New_York')
current_day = datetime.datetime.now(tz)

time_label = "Midday" if (current_day.hour <= 14) else "Evening"

label = current_day.strftime("%Y_%m_%d_") + time_label
model.save(f'{label}.keras')
print("saved model")

saved model


In [40]:
df1 = df.copy()
to_predict = df1.tail(window_length)
to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)
to_predict

<ipython-input-40-efb52fef0daf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)


,N01,N02,N03,N04,N05,N06,N07,N08,N09,N10,N11,N12,N13,N14,N15,N16,N17,N18,N19,N20
319,7,12,15,18,19,23,27,30,33,37,38,43,45,46,48,52,53,55,61,68
320,3,9,10,14,17,19,20,26,27,29,35,37,38,42,44,50,56,57,68,69
321,6,13,17,18,21,30,37,39,42,47,48,49,53,57,58,60,61,62,65,69
322,9,10,12,17,18,19,20,23,30,31,32,34,37,38,42,48,51,54,55,70


In [41]:
prediction = np.array(df.tail(1))

x_next = scaler.transform(to_predict)
y_pred = model.predict(np.array([x_next]))

temp = np.array(np.unique(scaler.inverse_transform(y_pred).astype(int)[0]))
combined_array = np.concatenate((temp, np.array(prediction[0])))
winning_numbers = np.bincount(combined_array)
winners = np.unique(combined_array)

uniques, uniq_idx, counts = np.unique(combined_array,return_index=True,return_counts=True)
duplicates = combined_array[uniq_idx[counts>=2] ]  # <--- Get duplicates

top_5_indices = np.argsort(y_pred[0], axis=0)[::-1][:5]

top_5_numbers = []

winning_indices = np.where(np.isin(prediction[0], duplicates))[0]

# Print the winning indices

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)

print("The top 5 indices:", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")
top_10_indices = np.argsort(y_pred[0], axis=0)[::-1][:10]
top_10_numbers = []

winning_indices = np.where(np.isin(prediction[0], duplicates))[0]

for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_10_numbers.append(number)

print("The top 10 indices:", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("The predicted numbers for the lottery game are:  ", temp)
print("The actual numbers in the last lottery game were:", prediction[0])
print("\n")
print("The winning indices:", winning_indices)
print("winners:", duplicates)
print(len(duplicates)/20)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - 5s 5s/step
The top 5 indices: [17 18 19  7  6]
The top 5 numbers: [59, 61, 66, 32, 29]


The top 10 indices: [17 18 19  7  6 16  4  5  8  9]
The top 10 numbers: [59, 61, 66, 32, 29, 57, 17, 26, 34, 37]


The predicted numbers for the lottery game are:   [ 2  5  8 13 18 21 25 29 31 34 36 39 42 47 50 53 58 65 67 69]
The actual numbers in the last lottery game were: [ 3  4 11 14 17 26 29 32 34 37 41 45 46 48 53 56 57 59 61 66]


The winning indices: [ 6  8 14]
winners: [29 34 53]
0.15


# Predict the next set of numbers

In [42]:
next = df.copy()
next = next.tail(window_length)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))
prediction = scaler.inverse_transform(y_next_pred).astype(int)
print("The predicted numbers for the lottery game:", temp)
print("\n")
top_5_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:5]
top_5_numbers = []
for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)

print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

winning_numbers = []
for index in winning_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    winning_numbers.append(number)

top_10_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:10]
top_10_numbers = []
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)
print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("winning indices", winning_indices)
print("winning numbers", winning_numbers)


1/1 [==============================] - 3s 3s/step
The predicted numbers for the lottery game: [ 2  5  8 13 18 21 25 29 31 34 36 39 42 47 50 53 58 65 67 69]


The top 5 indices [17 18 19  7  4]
The top 5 numbers: [64, 66, 68, 28, 17]


The top 10 indices [17 18 19  7  4  6  5  0  9  8]
The top 10 numbers: [64, 66, 68, 28, 17, 24, 20, 3, 33, 30]


winning indices [ 6  8 14]
winning numbers [24, 30, 44]


In [43]:
next = df.copy()
next = next.tail(7)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))

# Get the actual values of the top 10 predictions
prediction = scaler.inverse_transform(y_next_pred).astype(int)

# Get the indices of the top 10 predictions (change axis=0 if predictions are along a different axis)
top_10_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:10]
top_5_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:5]

top_10_numbers = []
top_5_numbers = []

# index [0,2,4] usually are correct
# Loop through each index in top_10_indices
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_5_numbers.append(number)

winning_numbers = []
for index in winning_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    winning_numbers.append(number)


print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("winning indices", winning_indices)
print("winning numbers", winning_numbers)
print("\n")

print("The predicted numbers:", prediction)


1/1 [==============================] - 0s 21ms/step
The top 5 indices [19  7 17  6  8]
The top 5 numbers: [71, 33, 65, 29, 34]


The top 10 indices [19  7 17  6  8 18  9  5  4 10]
The top 10 numbers: [71, 33, 65, 29, 34, 66, 37, 22, 19, 38]


winning indices [ 6  8 14]
winning numbers [29, 34, 47]


The predicted numbers: [[ 2  3  7 12 19 22 29 33 34 37 38 40 42 45 47 50 55 65 66 71]]
